In [1]:
using Pkg
# Pkg.activate(joinpath(@__DIR__, "../../"))
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging
using StaticArrays

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [2]:
# Choose f from: Rastringin, alpine1, Ackley
f = Ackley

# Choose n from: 2, 3, 4
n = 3

# Constants and Parameters
const a, b = 1, 1
const scale_factor = a / b
rand_center = zeros(Float64, n);
d = 10 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor 
                )

test_input(3, [0.0, 0.0, 0.0], 40, (0.1, 0.5), 0.002, (0.0, 0.0), 1.0, 1.0, 6, Globtim.Ackley)

Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [ ]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
@polyvar(x[1:n]); # Define polynomial ring 

In [4]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

Tracking 729 paths...   0%|▏                            |  ETA: 0:28:58



Tracking 729 paths... 100%|█████████████████████████████| Time: 0:00:05
  # paths tracked:                  729
  # non-singular solutions (real):  729 (125)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         729 (125)


125-element Vector{Vector{Float64}}:
 [0.9911840615254298, 0.9911840615254308, 0.9911840615254283]
 [-1.2954874092360902e-17, 0.9911840615254288, 0.9911840615254274]
 [-0.9911840615254316, 0.9911840615254267, 0.9911840615254288]
 [-0.5024836441161776, 0.9911840615254274, 0.9911840615254258]
 [0.5024836441161776, 0.9911840615254282, 0.9911840615254262]
 [0.9911840615254307, -3.3212824845121677e-16, 0.9911840615254295]
 [-1.0170985797104379e-17, -3.41157873845256e-16, 0.9911840615254286]
 [-0.9911840615254293, -3.2409520648058854e-16, 0.991184061525431]
 [-0.5024836441161776, -3.3501623567717305e-16, 0.99118406152543]
 [0.5024836441161777, -3.361078150499635e-16, 0.991184061525428]
 ⋮
 [-6.611672872058363e-18, -0.5024836441161781, 0.5024836441161773]
 [-0.9911840615254311, -0.502483644116178, 0.5024836441161772]
 [-0.5024836441161773, -0.502483644116178, 0.5024836441161774]
 [0.5024836441161776, -0.5024836441161781, 0.5024836441161774]
 [0.9911840615254317, 0.5024836441161775, 0.50248364

In [5]:
df_cheb = process_crit_pts(real_pts_cheb, f, TR)

Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,0.991184,0.991184,0.991184,-27.0066
2,-1.29549e-17,0.991184,0.991184,-28.0044
3,-0.991184,0.991184,0.991184,-27.0066
4,-0.502484,0.991184,0.991184,-7.75316
5,0.502484,0.991184,0.991184,-7.75316
6,0.991184,-3.32128e-16,0.991184,-28.0044
7,-1.0171e-17,-3.41158e-16,0.991184,-29.0022
8,-0.991184,-3.24095e-16,0.991184,-28.0044
9,-0.502484,-3.35016e-16,0.991184,-8.75094


In [6]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.5);

Processing point 1 of 125
Optimization has converged within bounds: ✓
Processing point 2 of 125
Optimization has converged within bounds: ✓
Processing point 3 of 125
Optimization has converged within bounds: ✓
Processing point 4 of 125
Optimization has converged within bounds: ✓
Processing point 5 of 125
Optimization has converged within bounds: ✓
Processing point 6 of 125
Optimization has converged within bounds: ✓
Processing point 7 of 125
Optimization has converged within bounds: ✓
Processing point 8 of 125
Optimization has converged within bounds: ✓
Processing point 9 of 125
Optimization has converged within bounds: ✓
Processing point 10 of 125
Optimization has converged within bounds: ✓
Processing point 11 of 125
Optimization has converged within bounds: ✓
Processing point 12 of 125
Optimization has converged within bounds: ✓
Processing point 13 of 125
Optimization has converged within bounds: ✓
Processing point 14 of 125
Optimization has converged within bounds: ✓
Processing poin

In [ ]:
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.05);

@info "" df_cheb

Globtim._TO

grid = TR.sample_range * generate_grid(n, GN, basis=:chebyshev);
new_grid = map(x -> x + p_center, grid);
values = map(f, grid); # Prepare level set data for specific level

using GLMakie
# fig = Globtim.create_level_set_visualization(error_func, new_grid, df_cheb, (0., 1000.))
fig = Globtim.plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb)
# display(fig)

